In [2]:
pip install jieba

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.
  Using cached jieba-0.42.1.tar.gz (19.2 MB)
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=0a5a28fecb4869120ec5b52b0b9f61a08ce67d17cd032ba1c3685cf78a466bba
  Stored in directory: c:\users\tony2\appdata\local\pip\cache\wheels\7d\74\cf\08c94db4b784e2c1ef675a600b7b5b281fd25240dcb954ee7e
Successfully built jieba

